In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import pandas
from sklearn import metrics # score functions and performance metrics
from sklearn.model_selection import train_test_split # split training and testing datasets
import keras # a high-level API to train and evaluate neural networks
from keras.models import Model, load_model # NN Model
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization # various layers
from keras.callbacks import ModelCheckpoint, EarlyStopping # callbacks used during the training

Common definitions

In [ ]:
class Samples:
    Signal_NonRes = -125
    Signal_Radion = [ -260, -270, -280, -300, -320, -340, -350,
                      -400, -450, -500, -550, -600, -650, -750, -800, -900 ]
    Data = 0
    TT = 1
    DY = 2
    Wjets= 3
    SM_Higgs = 4
    other_bkg = 5
    
class btag_wp:
    Loose = 0.5426
    Medium = 0.8484
    Tight = 0.9535

Load data. Select only branches with variables needed for the training in order to reduce the memory consumption.

In [ ]:
#path = "/eos/home-m/mgrippo/CMSDAS_2019_hh_bbtautau/anaTuples/"
path = "./anaTuples/"
channel = "tauTau"

branches = [ 'sample_id', 'region_id', 'm_sv', 'm_bb', 'm_ttbb_kinfit', 'csv_b*', 'weight' ]

with uproot.open(path+channel+"_tuple.root") as file:
    tree = file[channel]
    all_branches = [ b.decode('utf-8') for b in tree.allkeys() ]
    print("All available branches: ", all_branches)
    df = tree.arrays(branches, outputtype=pandas.DataFrame)
df.shape

### Training for nonresonant sample

Define branche to indicate siganl or background and fix variable ranges. For example, when kinfit is not converged, put -1, instead of the default value (which is std::numeric_limits<float>::lowest()).

In [ ]:
df['is_signal'] = pandas.Series((df.sample_id == Samples.Signal_NonRes).astype(int), df.index)
df['is_bkg'] = pandas.Series((df.sample_id > 0).astype(int), df.index)
df['m_ttbb_kinfit'] = pandas.Series(np.maximum(df.m_ttbb_kinfit, -1), df.index)

It is important to apply correct weights to give correct importance for different backgrounds. On the other hand, signal and background should have the same total weight.

In [ ]:
ml_weight = (df.weight * df.is_signal / np.sum(df.weight * df.is_signal) \
            + df.weight * df.is_bkg / np.sum(df.weight * df.is_bkg)) * df[(df.is_signal == 1) | (df.is_bkg == 1)].shape[0]
df['ml_weight'] = pandas.Series(ml_weight, df.index)

Define input branches 

In [ ]:
input_branches = ['m_sv', 'm_bb', 'm_ttbb_kinfit']
input_shape = (len(input_branches), )
print("Number of inputs: ", len(input_branches))

- select only signal and background events;
- shuffle the dataset
- split the original dataset into training and testing

In [ ]:
data = df[(df.is_signal == 1) | (df.is_bkg == 1)]
data_train, data_test = train_test_split(data.sample(frac=1.), test_size=0.2)
X_train = data_train[input_branches]
Y_train = data_train['is_signal']

Define neural network model and compile it.

In [ ]:
n_hidden_layers = 5 # number of hidden layers
n_neurons = len(input_branches) * 2 # number of neurons per layer
dropout_rate = 0.5 # fraction of the neurons that are randomly dropped before each training step

input_layer = Input(name="main_input", shape=input_shape) # Input layer

prev_layer = input_layer
for n in range(n_hidden_layers):
    # Fully connected dense layer. The weights are initialized with He uniform initializer, 
    # which samples weights from from an uniform distribution within [-limit, limit],
    # where limit is sqrt(6 / n_inputs).
    dense_layer = Dense(n_neurons, name="dense_%d" % n,
                        kernel_initializer='he_uniform')(prev_layer) 
    
    # Apply a batch normalization layer to reduce dependency on the statistical fluctuations between the batchs.
    # Batch normalization normalizes the output of a previous activation layer by subtracting
    # the batch mean and dividing by the batch standard deviation.
    layer = BatchNormalization(name="batch_norm_%d" % n)(dense_layer)
    
    # Activation layer that is needed to introduce non-linearity to the NN response.
    # Here ReLU activation function is used. ReLU(x) = max(0, x).
    activation_layer = Activation('relu', name="activation_%d" % n)(layer)
    
    # Layer that randomly drop part of the outputs before each training step.
    prev_layer = Dropout(dropout_rate, name="dropout_%d" % n)(activation_layer)
    
# Output layer and its activation. A signle output is used:
# for background-like events we want to have values closer to 0,
# while for signal-like events we want to have values closer to 1.
output_layer = Dense(1, name="dense_%d" % n_hidden_layers)(prev_layer)
softmax_output = Activation("sigmoid", name="main_output")(output_layer)

# Create model indicating input and output layers.
model = Model(input_layer, softmax_output, name='bbtautau_model')

# Optimizer that implements the Adam algorithm. See http://arxiv.org/abs/1412.6980
# lr indicates the learning rate of the algorithm.
# Here is the list of optimizers available in Keras: https://keras.io/optimizers/
opt = keras.optimizers.Adam(lr=1e-3)

# Compile the model. 
# Cross entropy is chosen as the loss function, while the accuracy is used as
# an additional metric for cross-check purposes.
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Train the network

In [ ]:
# Callback that stores the best network based on the validation loss.
checkpoint = ModelCheckpoint('best_network.hdf5', monitor='val_loss', save_best_only=True, save_weights_only=False)
# Callback that stops the training, if the selected quantity does not improve for the given number of epochs.
early_stopping = EarlyStopping('val_loss', patience=10)
# Fit the model and store the history.
# - 25% of the training set is used for the validation
# - each batch contains 10k events
# - total number of epochs is 1000, but should stop much earlier by the EarlyStopping callback
history = model.fit(X_train.values, Y_train.values, sample_weight=data_train.ml_weight.values,
                    callbacks=[checkpoint, early_stopping], validation_split=0.25,
                    batch_size=10000, epochs=1000, verbose=1)

Plot the loss for training and validation sets as a function of epoch number.

In [ ]:
plt.plot(history.epoch, history.history['loss'], history.history['val_loss'])
plt.legend(['training set', 'validation set'])
plt.title('Loss vs epoch')
plt.xlabel('epoch')
plt.ylabel('loss');

Load the best network and apply it to the test set.

In [ ]:
model = load_model('best_network.hdf5')
pred = model.predict(data_test[input_branches].values)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(data_test['is_signal'], pred[:, 0], sample_weight=data_test.ml_weight.values)
plt.plot(tpr, 1-fpr)
plt.title('ROC curve', fontsize=20)
plt.xlabel('Signal efficiency', fontsize=16)
plt.ylabel('Background rejection', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
metrics.roc_auc_score(data_test['is_signal'], pred[:, 0], sample_weight=data_test.ml_weight.values)

### Understand network performance
- Add more variables.
  - You can't use tau isolation, because it is used in the sideband definition.
  - If you use csv as an input, you should apply at least the loose btag working point,
    otherwise the agreement between data and MC is not guaranteed.
- Evaluate the performance in the signal region against all and against the main background (ttbar).
- Compare performance of the training that run only on the signal region events
  (less stat, but the parameter space is exact as in the final selection).
- Tune network parameters and see the effect on the performance:
  - number of layers
  - number of neurons per layer
  - decreas number of neurons in each consecutive layer
  - learning rate
  - other minimizers: SGD, NAdam
  - effect of dropout rate
  - effect of batch normalization:
    - train network with ReLU without the batch normalization
    - implement self-normalizing network using SELU activation.
      In that case the kernel_init should be lecun_normal and AlphaDropout should be used for dropout.
- Use S/sqrt(S+B) estimator from the previous exercise to evaluate if the additional cut on the NN output improves the significance. 

### Parametrized learning
- Try to apply the training procedure established above discriminate the resonant signal.
- Add gen mass as parameter and see if performance has improved.
- Given one mass hypothesis, see how performance changes for the signal samples with the different mX.
- Add production_mode parameter to be able to train both resonant and non-resonant together.
- (advanced) add channel parameter, so all the channels can be trained together.

Some code snippets are below

In [ ]:
# Now is_signal = 1 for all Radion samples
df['is_signal'] = pandas.Series(((df.sample_id < 0) & (df.sample_id != Samples.Signal_NonRes)).astype(int), df.index)
df['is_bkg'] = pandas.Series((df.sample_id > 0).astype(int), df.index)

In [ ]:
# new ml_weight. making sure that all mass points have the same total weight.
ml_weight = df.weight * df.is_bkg / np.sum(df.weight * df.is_bkg)
for mX in Samples.Signal_Radion:
    ml_weight += df.weight * (df.sample_id == mX).astype(int) / np.sum(df.weight * (df.sample_id == mX).astype(int)) \
                 / len(Samples.Signal_Radion)
ml_weight *= df[(df.is_signal == 1) | (df.is_bkg == 1)].shape[0]
df['ml_weight'] = pandas.Series(ml_weight, df.index)

In [ ]:
# Create branch with mX at the gen level to be used as an input.
# Set it to a random value for the backgrounds.
mX = -df.sample_id * df.is_signal - df.is_bkg * np.random.choice(Samples.Signal_Radion, size=df.shape[0]) 
df['mX'] = pandas.Series(mX, df.index)

In [ ]:
# add mX to the list of the input branches
input_branches = [ 'mX', ... ]
input_shape = (len(input_branches), )
print("Number of inputs: ", len(input_branches))

In [ ]:
# split data in training and testing as before
data = df[(df.is_signal == 1) | (df.is_bkg == 1)]
data_train, data_test = train_test_split(data.sample(frac=1.), test_size=0.2)
X_train = data_train[input_branches]
Y_train = data_train['is_signal']

In [ ]:
# define the NN model and compile it
...


In [ ]:
# train the model
...

In [ ]:
# plot losses
...

In [ ]:
# load the best model
#model = load_model('best_param_network.hdf5')

In [ ]:
# Evaluate the model on the test set for each mX hypothesis.
scores = []
for sample_id in Samples.Signal_Radion:
    mX = -sample_id
    data_test_mX = data_test[(data_test.sample_id == sample_id) | (data_test.sample_id > 0)].copy()
    data_test_mX['mX'] = pandas.Series(np.ones(data_test_mX.shape[0]) * mX, data_test_mX.index)
    pred = model.predict(data_test_mX[input_branches].values)
    fpr, tpr, thresholds = metrics.roc_curve(data_test_mX['is_signal'], pred[:, 0], sample_weight=data_test_mX.ml_weight.values)
    plt.plot(tpr, 1-fpr)
    plt.title('ROC curve for mX = {} GeV'.format(mX), fontsize=20)
    plt.xlabel('Signal efficiency', fontsize=16)
    plt.ylabel('Background rejection', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    score = metrics.roc_auc_score(data_test_mX['is_signal'], pred[:, 0], sample_weight=data_test_mX.ml_weight.values)
    print('mX = {} GeV, score = {}'.format(mX, score))
    scores.append(score)
    plt.show()

In [ ]:
# plot auc score as a function of mX
plt.plot(-np.array(Samples.Signal_Radion), scores)
plt.title('auc score vs. mX')
plt.xlabel('mX (GeV)')
plt.ylabel('auc score')